### IO and utilities

In [1]:
import os
from dataclasses import dataclass
import re

def read_file(path: str) -> list[str]:
    with open(os.path.join(os.getcwd(), path), "r") as f:
        return [line.strip() for line in f.readlines()]
    
StringMatrix = list[str]

def read_file_as_matrix(path: str) -> StringMatrix:
    with open(os.path.join(os.getcwd(), path), "r") as f:
        return [line.strip() for line in f.readlines()]


# Day 1: Trebuchet?!


In [11]:
def getnumber(line: str) -> int:
    numbers = [c for c in line if c.isdigit()]
    return int(numbers[0] + numbers[-1])

day1_1 = read_file("day1/1.txt")
print(sum([getnumber(l) for l in day1_1]))


54630


Part 2, search each line for the first digit or word, the search from back after first digit or word.

In [42]:
def test(substr: str) -> str:
    numbers = {
        'one': '1',
        'two': '2',
        'three': '3',
        'four': '4',
        'five': '5',
        'six': '6',
        'seven': '7',
        'eight': '8',
        'nine': '9'
    }

    for word, digit in numbers.items():
        if word in substr:
            return digit

    return ''

def find_first_digit(inline: str, reverse = False) -> str:
    buffer = ''
    line = inline[::-1] if reverse else inline

    for c in line:
        if c.isdigit():
            return c

        if reverse:
            buffer = c + buffer
        else:
            buffer = buffer + c

        candidate = test(buffer)
        if candidate:
            return candidate

def replace_text(line: str) -> str:
    return find_first_digit(line) + find_first_digit(line, True)
   
day1_2 = read_file("day1/2.txt")
parsed = [int(replace_text(l)) for l in day1_2]
print(sum(parsed))

54770


# Day 2: Cube Conundrum

### Data representation and parsing

In [ ]:
@dataclass
class GameSet():
    red: int
    green: int
    blue: int

@dataclass
class Game():
    id: int
    sets: list[GameSet]

def parse_set(s: str) -> GameSet:
    gameset = GameSet(0,0,0)
    cubesstr = s.split(',')

    for cube in cubesstr:
        n = int(re.search('\d+', cube).group())
        if 'red' in cube:
            gameset.red = n
        elif 'green' in cube:
            gameset.green = n
        else:
            gameset.blue = n
    
    return gameset


def parse(line: str) -> Game:
    g = Game(-1, [])
    
    gamesrt, setsstr = line.split(':')
    g.id = int(gamesrt[4:])
    g.sets = [parse_set(s) for s in setsstr.split(';')]
    return g

day2_1 = read_file('day2/1.txt')
games = [parse(l) for l in day2_1]

### Part 1: Finding possible games

In [54]:

def max_set(a: GameSet, b: GameSet) -> GameSet:
    return GameSet(max(a.red, b.red), max(a.green, b.green), max(a.blue, b.blue))

def possible(game: Game, max_allowed: GameSet) -> bool:
    game_max = GameSet(0,0,0)

    for gameset in game.sets:
        game_max = max_set(game_max, gameset)

    return game_max.red <= max_allowed.red and game_max.green <= max_allowed.green and game_max.blue <= max_allowed.blue

valid_game_ids = [g.id for g in games if possible(g, GameSet(12, 13, 14))]
print(sum(valid_game_ids))


2447


### Part 2: Finding mimimum possible cubes and summing power of these sets

In [55]:
def minimum_possible_cubes(game: Game) -> GameSet:
    game_min = GameSet(0,0,0)

    for gameset in game.sets:
        game_min = max_set(game_min, gameset)

    return game_min

def game_power(game: Game) -> int:
    s = minimum_possible_cubes(game)
    return s.red * s.green * s.blue

print(sum([game_power(g) for g in games]))

56322


# Day 3: Gear Ratios

### Utils

In [68]:

@dataclass
class Coord():
    row: int
    col: int

def getAdjecent(coord: Coord, total_rows: int, total_cols: int) -> list[Coord] :
    result = []
    
    neighbouring_left = coord.col > 0
    neighbouring_right = coord.col < total_cols
    neighbouring_up = coord.row > 0
    neighbouring_down = coord.row < total_rows

    if neighbouring_up:
        if neighbouring_left:
            result.append(Coord(coord.row - 1, coord.col - 1))
        
        result.append(Coord(coord.row - 1, coord.col))

        if neighbouring_right:
            result.append(Coord(coord.row - 1, coord.col + 1))
    
    if neighbouring_left:
        result.append(Coord(coord.row, coord.col - 1))

    if neighbouring_right:
        result.append(Coord(coord.row, coord.col + 1))

    if neighbouring_down:
        if neighbouring_left:
            result.append(Coord(coord.row + 1, coord.col - 1))
        
        result.append(Coord(coord.row + 1, coord.col))

        if neighbouring_right:
            result.append(Coord(coord.row + 1, coord.col + 1))

    return result

def get_char(data: StringMatrix, coord: Coord) -> str:
    return data[coord.row][coord.col]

def is_digit(data: StringMatrix, coord: Coord) -> bool:
    return get_char(data, coord).isdigit()

def getIntIndex(data: StringMatrix, coord: Coord) -> (int,int):
    found_first = False
    found_last = False 

    f_index = coord.col
    l_index = coord.col

    while not found_first:
        candidate = f_index - 1
        if candidate >= 0 and data[coord.row][candidate].isdigit():
            f_index = candidate
        else:
            found_first = True

    while not found_last:
        candidate = l_index + 1
        if candidate < len(data[coord.row]) and data[coord.row][candidate].isdigit():
            l_index = candidate
        else:
            found_last = True

    return f_index, l_index

def remove_data(data: StringMatrix, row, begin, end):
    current = data[row]
    data[row] = current[:begin] + 'x' * (end + 1 - begin) + current[end+1:]



### Part 1

In [63]:
def is_symbol(data: StringMatrix, coord: Coord) -> bool:
    symbols = ["+","*","#","$","%","@","#","&","=","-",'/']
    return get_char(data, coord) in symbols

result = 0
data = read_file_as_matrix("day3/input.txt")
for row in range(len(data)):
    for col in range(len(data[row])):
        c = Coord(row, col)
        if is_symbol(data, c):
            adjecents = getAdjecent(c, len(data), len(data[row]))

            for a in adjecents:
                if is_digit(data, a):
                    first, last = getIntIndex(data, a)
                    number = int(data[a.row][first:last+1])
                    result += number
                    remove_data(data, a.row, first, last)

print(result)

4361


### Part 2 - find gears

In [70]:
def is_gear(data: StringMatrix, coord: Coord) -> bool:
    return get_char(data, coord) == '*'

result = 0
data = read_file_as_matrix("day3/input.txt")
for row in range(len(data)):
    for col in range(len(data[row])):
        c = Coord(row, col)
        if is_gear(data, c):
            adjecents = getAdjecent(c, len(data), len(data[row]))
            
            numbers = []

            data_before = data

            for a in adjecents:
                if is_digit(data, a):
                    first, last = getIntIndex(data, a)
                    number = int(data[a.row][first:last+1])
                    numbers.append(number)
                    remove_data(data, a.row, first, last)

            if len(numbers) == 2:
                result += numbers[0] * numbers[1]
            else: 
                # No gear! Restore data
                data = data_before
print(result)

84900879


# Day 4: Scratchcards

### Part 1

In [17]:
@dataclass
class Card():
    id: int
    winning_numbers: set[int]
    numbers: set[int]
    
def parse_card(line: str) -> Card:
    card_nr_str, rest = line.split(":")
    c = Card(int(card_nr_str[5:].strip()), [], [])

    winning_str, numbers_str = rest.split("|")
    c.winning_numbers = {int(n.strip()) for n in winning_str.split()}
    c.numbers = {int(n.strip()) for n in numbers_str.split()}

    return c

def worth(c: Card) -> int:
    wins = c.numbers.intersection(c.winning_numbers)
    return 0 if len(wins) == 0 else pow(2, len(wins) - 1)

data = read_file("day4/input.txt")
cards = [parse_card(l) for l in data]
winnings = [worth(c) for c in cards]
print(sum(winnings))


21568


### Part 2

In [23]:
@dataclass
class Card():
    id: int
    winning_numbers: set[int]
    numbers: set[int]
    copies: int
    
def parse_card(line: str) -> Card:
    card_nr_str, rest = line.split(":")
    c = Card(int(card_nr_str[5:].strip()), [], [], 1)

    winning_str, numbers_str = rest.split("|")
    c.winning_numbers = {int(n.strip()) for n in winning_str.split()}
    c.numbers = {int(n.strip()) for n in numbers_str.split()}

    return c

def wins(c: Card) -> int:
    wins = len(c.numbers.intersection(c.winning_numbers))
    return range(c.id, c.id+wins)
      

data = read_file("day4/input.txt")
cards = [parse_card(l) for l in data]
for card in cards:
    new_copies = wins(card)
    for id in new_copies:
        if id < len(cards):
            cards[id].copies += card.copies

total = sum([c.copies for c in cards])
print(total)

30
